In [1]:
import tensor_comprehensions as tc
import torch
import numpy as np
import matplotlib.pyplot as plt

import os
import sys
import logging
import time

In [2]:
tc.decode('cache/tbmm_500_26_72_26.tc.options')

In [3]:
# try doing tbmm using the options provided in the paper

options = tc.Options('naive')

In [4]:
print(options)

In [10]:
options.tile([1,1])
options.mapToThreads([8, 14, 14])
options.mapToBlocks([32, 32, 3])
options.useSharedMemory(True)
options.unrollCopyShared(True)
options.unroll(256)


In [6]:
gconv_lang = tc.database['group_convolution']['lang']

gconv = tc.define(gconv_lang, name='group_convolution')

In [12]:
# try the sizes given in the paper
# N = 32, G = 32, F = 16, C = 16, W = 14, H = 14

I = torch.randn(128, 16, 16, 10, 10).cuda()
W1 = torch.randn(16, 16, 16, 3, 3).cuda()
B = torch.randn(16, 16).cuda()

out = gconv(I, W1, B, options=options)

[ERROR]: Caught Exception: Error at: /opt/conda/conda-bld/tensor_comprehensions_1520457708651/work/src/core/rtc.cc:183: CUDA_ERROR_INVALID_VALUE


In [11]:
# test performance using TC
torch.cuda.synchronize()

timings = np.zeros(1000)

for i in range(1000):
    start = time.perf_counter()
    output = gconv(I, W1, B, options=options)
    torch.cuda.synchronize()
    timings[i] = time.perf_counter() - start

total_elapsed = np.sum(timings)
print('total time: ' + str(total_elapsed))

# get percentile statistics
percentiles = np.percentile(timings, [0, 50, 90])
print(percentiles)




total time: 1.106699081
[ 0.0010849   0.00109942  0.00111415]
